# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
#from train import clean_data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Breastcancer_automl'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-142737
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-142737


In [4]:
ds = TabularDatasetFactory.from_delimited_files(path="https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv")

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
cpu_cluster_name = "cpucluster-aml"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
#training_data, validation_data = d.random_split(percentage=0.8, seed=1)

In [8]:
automl_settings = {
    "experiment_timeout_minutes" :30,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'average_precision_score_weighted',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    n_cross_validations=5,
    task="classification",
    primary_metric="average_precision_score_weighted",
    compute_target=cpu_cluster,
    training_data=ds,
    label_column_name="Classification",
    max_cores_per_iteration=-1,
    enable_onnx_compatible_models=True
    )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(config = automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpucluster-aml with default configuration
Running on remote compute: cpucluster-aml


Experiment,Id,Type,Status,Details Page,Docs Page
Breastcancer_automl,AutoML_1421a1dd-b7e4-4366-a79a-f5cf3c837060,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
         3   RobustScaler ExtremeRandomTrees                0:05:25       0.8349    0.8722
         4   MinMaxScaler ExtremeRandomTrees                0:01:03       0.8472    0.8722
         5   StandardScalerWrapper RandomForest             0:00:46       0.7800    0.8722
         6   MinMaxScaler ExtremeRandomTrees                0:00:50       0.8191    0.8722
         7   StandardScalerWrapper KNN                      0:00:52       0.6373    0.8722
         8   MinMaxScaler ExtremeRandomTrees                0:01:11       0.8298    0.8722
         9   RobustScaler RandomForest                      0:00:46       0.8279    0.8722
        10   StandardScalerWrapper RandomForest             0:00:52       0.8093    0.8722
        11   StandardScalerWrapper ExtremeRandomTrees       0:00:52       0.8497    0.8722
        12   RobustScaler ExtremeRandomTrees                0:00:55       0.8413    0.8722
        13   Min

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

model_ml = best_run.register_model(model_name='Cervical_Cancer_prediction_Biopsy_auto_ml', model_path='./')

Run(Experiment: Breastcancer_automl,
Id: AutoML_1421a1dd-b7e4-4366-a79a-f5cf3c837060_23,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.15052631578947367,
                                                                                                  min_weight_fraction_leaf

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service